In [1]:
# %pip install scikit-learn
# %pip install matplotlib
# %pip install scipy
# %pip install statsmodels

In [2]:
import pandas as pd
# Set max columns to display
pd.set_option('display.max_columns', None)

import numpy as np
from sklearn import preprocessing as pre
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

C:\Users\austinsh\AppData\Local\Temp\ipykernel_10836\3666630488.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# # Importing CSV files
# df_CDunit = pd.read_csv('https://raw.githubusercontent.com/saust1/Project-OptiC4/main/1%20Preprocess/Continuous%20Data/cont_554Data_clean.csv')
# df_AlCon = pd.read_csv('https://raw.githubusercontent.com/saust1/Project-OptiC4/main/1%20Preprocess/Continuous%20Data/cont_425Data_clean.csv')
# df_FB554 = pd.read_csv('https://raw.githubusercontent.com/saust1/Project-OptiC4/main/1%20Preprocess/Continuous%20Data/cont_unitData_clean.csv')


In [4]:


# bordeCode directory
df_CDunit = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\II Data\2 Preprocessing\Continuous Data\cont_unitData_clean.csv')
df_AlCon = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\II Data\2 Preprocessing\Continuous Data\cont_425Data_clean.csv')
df_FB554 = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\II Data\2 Preprocessing\Continuous Data\cont_554Data_clean.csv')

In [5]:
print(df_CDunit.describe())
print(df_AlCon.describe())
print(df_FB554.describe())


           AYC55580       DI55102       DI55152       DI55580       FC42428  \
count  40127.000000  40127.000000  40127.000000  40127.000000  40127.000000   
mean      11.106674      0.944623      0.932484      0.995797  36811.095494   
std        4.087796      0.051776      0.029015      0.046053   4311.627143   
min        0.000000      0.800007      0.837832      0.853967  19249.500000   
25%        9.924740      0.911098      0.913068      0.965977  35739.700000   
50%       10.786900      0.949496      0.932305      0.997192  38146.000000   
75%       13.196700      0.984039      0.951719      1.029190  39516.800000   
max       20.000000      1.052060      1.020350      1.098610  49959.600000   

            FC52018       FC55003       FC55009       FC55102       FC55152  \
count  40127.000000  40127.000000  40127.000000  40127.000000  40127.000000   
mean   32866.825818   5994.183724    845.358602  44806.974341  40473.976083   
std     6142.280506    824.261804    610.986652   5

In [6]:
print("Data type for 'Date' column in df_CDunit:", df_CDunit['Date'].dtypes)
print("Data type for 'Date' column in df_FB554:", df_FB554['Date'].dtypes)
print("Data type for 'Date' column in df_AlCon:", df_AlCon['Date'].dtypes)


Data type for 'Date' column in df_CDunit: object
Data type for 'Date' column in df_FB554: object
Data type for 'Date' column in df_AlCon: object


In [7]:
df_CDunit['Date'] = pd.to_datetime(df_CDunit['Date'], errors='coerce')
df_FB554['Date'] = pd.to_datetime(df_FB554['Date'], errors='coerce')
df_AlCon['Date'] = pd.to_datetime(df_AlCon['Date'], errors='coerce')


In [8]:
print("Data type for 'Date' column in df_CDunit:", df_CDunit['Date'].dtypes)
print("Data type for 'Date' column in df_FB554:", df_FB554['Date'].dtypes)
print("Data type for 'Date' column in df_AlCon:", df_AlCon['Date'].dtypes)


Data type for 'Date' column in df_CDunit: datetime64[ns]
Data type for 'Date' column in df_FB554: datetime64[ns]
Data type for 'Date' column in df_AlCon: datetime64[ns]


In [9]:
print(df_CDunit.columns)
print(df_FB554.columns)
print(df_AlCon.columns)


Index(['Date', 'AYC55580', 'DI55102', 'DI55152', 'DI55580', 'FC42428',
       'FC52018', 'FC55003', 'FC55009', 'FC55102', 'FC55152', 'FC55552',
       'FC55555', 'FC55569', 'FC55576', 'FFC55553', 'FFC55555', 'FYC55553',
       'II52554', 'LC52572', 'LC55553', 'LC55555', 'LC55557', 'LC55568',
       'LC90366', 'LC90368', 'PI55004', 'PI55020', 'PI55560', 'TC52015',
       'TC55552', 'TC55553', 'TC55555', 'TC55566', 'TI40050', 'TI52014',
       'TI55013', 'TI55014', 'TI55015', 'TI55016', 'TI55017', 'TI55021',
       'TI55023', 'VI52558B'],
      dtype='object')
Index(['Date', 'Butanol', 'Decanol', 'Ethanol', 'Hexanol', 'Octanol'], dtype='object')
Index(['Date', '425_pct_Al', 'Al2O3', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O',
       'HydWtr_pct_Ammonia', 'C4_pct_Hex', 'HydWtr_Na2O'],
      dtype='object')


In [10]:
def apply_rolling_average_to_df(df, rolling_size):
    # Ensure 'Date' is the index if it's not already
    if df.index.name != 'Date':
        df = df.set_index('Date')

    # Apply rolling average to all columns
    rolled_df = df.rolling(window=rolling_size, min_periods=1).mean()

    # Reset index to make 'Date' a column again
    rolled_df = rolled_df.reset_index()

    return rolled_df



In [11]:
def apply_time_shift_by_hours(df, shift_hours):
    """
    Shifts the DataFrame's datetime index by the specified number of hours.

    :param df: DataFrame with 'Date' as its datetime index or column.
    :param shift_hours: Number of hours to shift. Can be positive (forward) or negative (backward).
    :return: Shifted DataFrame.
    """
    # Convert 'Date' to datetime and set as index if it's not already
    if df.index.name != 'Date':
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df = df.set_index('Date')

    # Ensure the index is a DatetimeIndex
    df.index = pd.to_datetime(df.index)

    # Shift the DataFrame's index by the specified number of hours
    df.index = df.index + pd.Timedelta(hours=shift_hours)

    return df

In [12]:
# # Usage Examples
# shift_hours_AlCon = 1  # Negative shift for df_AlCon (e.g., -5 hours backward)
# shift_hours_FB554 = 5   # Positive shift for df_FB554 (e.g., 5 hours forward)

# shifted_df_AlCon = apply_time_shift_by_hours(df_AlCon, shift_hours_AlCon)
# print("Shifted df_AlCon:")
# print(shifted_df_AlCon.head())

# shifted_df_FB554 = apply_time_shift_by_hours(df_FB554, shift_hours_FB554)
# print("\nShifted df_FB554:")
# print(shifted_df_FB554.head())

In [13]:
def join_df_FB554_to_df_CDunit(df_CDunit, df_FB554):
    # Ensure 'Date' columns are datetime objects and sort DataFrames
    df_CDunit['Date'] = pd.to_datetime(df_CDunit['Date'], errors='coerce')
    df_FB554['Date'] = pd.to_datetime(df_FB554['Date'], errors='coerce')

    df_CDunit = df_CDunit.dropna(subset=['Date']).sort_values('Date')
    df_FB554 = df_FB554.dropna(subset=['Date']).sort_values('Date')

    # Perform merge_asof
    combined_df = pd.merge_asof(df_FB554, df_CDunit, on='Date', direction='nearest')

    return combined_df

def join_df_AlCon_to_combined_df(combined_df, df_AlCon):
    # Ensure 'Date' columns are datetime objects and sort DataFrames
    combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')
    df_AlCon['Date'] = pd.to_datetime(df_AlCon['Date'], errors='coerce')

    combined_df = combined_df.dropna(subset=['Date']).sort_values('Date')
    df_AlCon = df_AlCon.dropna(subset=['Date']).sort_values('Date')

    # Perform merge_asof
    combined_df_all = pd.merge_asof(df_AlCon, combined_df, on='Date', direction='nearest')
    
    return combined_df_all


In [14]:
print("Data type for 'Date' column in df_CDunit:", df_CDunit['Date'].dtypes)
print("Data type for 'Date' column in df_FB554:", df_FB554['Date'].dtypes)
print("Data type for 'Date' column in df_AlCon:", df_AlCon['Date'].dtypes)



Data type for 'Date' column in df_CDunit: datetime64[ns]
Data type for 'Date' column in df_FB554: datetime64[ns]
Data type for 'Date' column in df_AlCon: datetime64[ns]


In [15]:
def apply_negative_shift_hours(shift_hours):
    return [-hour for hour in shift_hours]

# Rolling sizes ranges
rolling_size_CDunit = range(4, 11, 2)# [8]  # Even rolling sizes from 4 to 10 range(4, 11, 2)
rolling_size_FB554 = range(4, 11, 2)# [4]  # Even rolling sizes from 4 to 10 range(4, 11, 2)
rolling_size_AlCon = range(2, 31, 2)#  [2]  # Even rolling sizes from 2 to 30 range(2, 31, 2) 

# Shift hours ranges
shift_hours_AlCon = apply_negative_shift_hours(range(2, 3, 2))  # Negative shifts from -2 to -8
shift_hours_FB554 = range(2, 8, 2)   # Positive shifts from 2 to 8

# Precompute rolling averages for each DataFrame and each rolling size
precomputed_rolls = {
    "CDunit": {size: apply_rolling_average_to_df(df_CDunit, size) for size in rolling_size_CDunit},
    "FB554": {size: apply_rolling_average_to_df(df_FB554, size) for size in rolling_size_FB554},
    "AlCon": {size: apply_rolling_average_to_df(df_AlCon, size) for size in rolling_size_AlCon}
}

results = pd.DataFrame()

## Modified process_data function
def process_data():
    iteration_count = 0
    results = pd.DataFrame(columns=['Iteration', 'Rolling Sizes CDunit', 'Rolling Sizes FB554', 'Rolling Sizes AlCon',
                                    'Shift Hours AlCon', 'Shift Hours FB554', 'R-squared', 'Adj R-Squared', 
                                    'F-statistic', 'Prob (F-statistic)'])

    for size_CDunit in rolling_size_CDunit:
        for size_FB554 in rolling_size_FB554:
            for size_AlCon in rolling_size_AlCon:
                for shift_hour_AlCon in shift_hours_AlCon:
                    for shift_hour_FB554 in shift_hours_FB554:
                        iteration_count += 1

                        # Retrieve rolled dataframes
                        rolled_df_CDunit = precomputed_rolls["CDunit"][size_CDunit]
                        rolled_df_FB554 = precomputed_rolls["FB554"][size_FB554]
                        rolled_df_AlCon = precomputed_rolls["AlCon"][size_AlCon]

                        # Combine df_CDunit and df_FB554 to create combined_df
                        combined_df = join_df_FB554_to_df_CDunit(rolled_df_CDunit, rolled_df_FB554)

                        # Combine combined_df with rolled_df_AlCon to create combined_df_all
                        combined_df_all = join_df_AlCon_to_combined_df(combined_df, rolled_df_AlCon)

                        # Drop 'Date' column before modeling
                        combined_df_all = combined_df_all.drop(columns=['Date'], errors='ignore')

                        # Splitting into train and test
                        X = combined_df_all.drop('Butanol', axis=1)
                        y = combined_df_all['Butanol']
                        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                        # Train model
                        model = sm.OLS(y_train, X_train).fit()

                        # Store the results instead of printing
                        iteration_results = {
                            'Iteration': iteration_count,
                            'Rolling Sizes CDunit': size_CDunit,
                            'Rolling Sizes FB554': size_FB554,
                            'Rolling Sizes AlCon': size_AlCon,
                            'Shift Hours AlCon': shift_hour_AlCon,
                            'Shift Hours FB554': shift_hour_FB554,
                            'R-squared': model.rsquared,
                            'Adj R-Squared': model.rsquared_adj,
                            'F-statistic': model.fvalue,
                            'Prob (F-statistic)': model.f_pvalue
                        }
                        # 1 results = results.append(iteration_results, ignore_index=True)?
                        # 2 results = pd.concat([results, iteration_results], ignore_index=True)
                        # iteration_results_df = pd.DataFrame([iteration_results])
                        # results = pd.concat([results], ignore_index=True) #, iteration_results_df
                        # Correctly convert iteration results to a DataFrame and concatenate
                        iteration_results_df = pd.DataFrame([iteration_results])
                        results = pd.concat([results, iteration_results_df], ignore_index=True)




                        # Print only the iteration count
                        print(f"Iteration: {iteration_count}")

    return results

# Call the function to process and evaluate the data
model_results = process_data()

C:\Users\austinsh\AppData\Local\Temp\ipykernel_10836\736028239.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, iteration_results_df], ignore_index=True)


Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
I

In [16]:
model_results

,Iteration,Rolling Sizes CDunit,Rolling Sizes FB554,Rolling Sizes AlCon,Shift Hours AlCon,Shift Hours FB554,R-squared,Adj R-Squared,F-statistic,Prob (F-statistic)
0,1,4,4,2,-2,2,0.798301,0.797955,2306.076059,0.0
1,2,4,4,2,-2,4,0.798301,0.797955,2306.076059,0.0
2,3,4,4,2,-2,6,0.798301,0.797955,2306.076059,0.0
3,4,4,4,4,-2,2,0.798396,0.798050,2307.438857,0.0
4,5,4,4,4,-2,4,0.798396,0.798050,2307.438857,0.0
...,...,...,...,...,...,...,...,...,...,...
715,716,10,10,28,-2,4,0.823320,0.823017,2715.139606,0.0
716,717,10,10,28,-2,6,0.823320,0.823017,2715.139606,0.0
717,718,10,10,30,-2,2,0.823329,0.823026,2715.313052,0.0
718,719,10,10,30,-2,4,0.823329,0.823026,2715.313052,0.0


In [18]:

# Looks like I  did this to consolidate 2 pages but ran into more problems than I was fixing.

# # Make sure to have your DataFrames: df_CDunit, df_FB554, df_AlCon ready

# # Apply the specific rolling average directly
# rolled_df_CDunit = apply_rolling_average_to_df(df_CDunit, 8)
# rolled_df_FB554 = apply_rolling_average_to_df(df_FB554, 4)
# rolled_df_AlCon = apply_rolling_average_to_df(df_AlCon, 2)

# # Apply the specific time shifts directly
# # Note: Since you're applying a negative shift to AlCon and a positive shift to FB554, ensure this logic is correctly handled in your apply_time_shift_by_hours function
# rolled_df_AlCon_shifted = apply_time_shift_by_hours(rolled_df_AlCon, -1) # apply_negative_shift_hours already applied in specifying -1
# rolled_df_FB554_shifted = apply_time_shift_by_hours(rolled_df_FB554, 1)

# # Merge the rolled and shifted DataFrames accordingly
# combined_df = join_df_FB554_to_df_CDunit(rolled_df_CDunit, rolled_df_FB554_shifted)
# combined_df_all = join_df_AlCon_to_combined_df(combined_df, rolled_df_AlCon_shifted)

# # Now, combined_df_all is the DataFrame you're interested in


In [20]:
# Save DataFrame to CSV file in the same directory as the Jupyter Notebook
# model_results.to_csv('model_results.csv', index=False)

model_results.to_csv(r'C:\Users\austinsh\Project-OptiC4\II Data\2 Preprocessing\Merge Data\iterative_adjR2_scan.csv', index=False)

# combined_df_all.to_csv(r'C:\Users\austinsh\Project-OptiC4\1 Preprocess\Merge Data\dataset_for_EDA.csv', index=False)
